## Lab | Creating First Prototype 

In [1]:
from bs4 import BeautifulSoup
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
url = "https://www.popvortex.com/music/charts/top-100-songs.php"

In [3]:
response = requests.get(url)

ConnectionError: HTTPSConnectionPool(host='www.popvortex.com', port=443): Max retries exceeded with url: /music/charts/top-100-songs.php (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fe5d4533c40>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [4]:
response.status_code

NameError: name 'response' is not defined

In [ ]:
len(response.content)

In [ ]:
soup = BeautifulSoup(response.content, "html.parser")

In [ ]:
soup

In [ ]:
soup.select("p.title-artist")

In [ ]:
soup.select("p.title-artist cite.title")[0].get_text()

In [ ]:
soup.select("p.title-artist em.artist")[0].get_text()

In [ ]:
#initialize empty lists
song_title = []
artist = []


# define the number of iterations of our for loop
# by checking how many elements are in the retrieved result set
# (this is equivalent but more robust than just explicitly defining 250 iterations)
num_iter = len(soup.select("p.title-artist"))

song_list = soup.select("p.title-artist cite.title")
artist_list = soup.select("p.title-artist em.artist")
# iterate through the result set and retrive all the data
for i in range(num_iter):
    song_title.append(song_list[i].get_text())
    artist.append(artist_list[i].get_text())


print(song_title)
print(artist)

In [ ]:
# each list becomes a column
Top100 = pd.DataFrame({"song_title":song_title,
                       "artist":artist
                      })
#we provide a dictionary where the key is the name of the column, and values are the values inside the column

In [ ]:
Top100.head(50)

In [ ]:
import random

In [ ]:
#https://albertoviciana.medium.com/my-first-song-recommender-96994af6d6bc

#we found code for this lab above

In [ ]:
def song_recommender():
    random_song=random.choice(Top100['song_title'])
    user_input= input("Please type in your favorite hit song, capitalizing each word (example: 'Heat Waves'): ")
    
    
    if user_input not in list(Top100['song_title']):
        print("I don't have a recommendation for you.")
    else:
        print("If you like the song " + "'" + user_input + "'" + " then listen to this next: " + "'" + random_song + "'")

In [ ]:
song_recommender()

# Lab | API wrappers - Create your collection of songs & audio features

To move forward with the project, you need to create a collection of songs 
with their audio features - as large as possible!
An idea for collecting as many songs as possible is to start with 
all the songs of a big, diverse playlist and then go to every artist present
in the playlist and grab every song of every album of that artist. 
The amount of songs you'll be collecting per playlist will grow exponentially!

Spotify objects are identified by either a "url", a "uri" or an "id".

The id is an alphanumeric code, and it's the nuclear part of the identifier.

The uri contains "spotify:track" before the id. An uri is useful because it can be searched manually in the Spotify app.

The url is a link to the song on the Spotify web player.

In [ ]:
#!pip install spotipy

#### Authentification

In [5]:
import spotipy   #the library
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentials
#sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id='<your client id here>',
#                                                           client_secret='<your client secret here>'))

#### storing secrets 

In [6]:
secrets_file = open("/Users/hollydalton/Downloads/secrets.txt","r")

In [7]:
string = secrets_file.read()

In [ ]:
string

In [ ]:
string.split('\n')

In [10]:
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1]

In [ ]:
secrets_dict

#### authentication with secrets

In [12]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['cid'],
                                                           client_secret=secrets_dict['csecret']))

Handling large playlists

In [13]:
playlist = sp.user_playlist_tracks("spotify", "1G8IpkZKobrIlXcVPoSIuf")

In [14]:
playlist["total"]

10000

In [15]:
playlist['next']  #gives next page of songs

'https://api.spotify.com/v1/playlists/1G8IpkZKobrIlXcVPoSIuf/tracks?offset=100&limit=100&additional_types=track'

In [16]:
sp.next(playlist)

{'href': 'https://api.spotify.com/v1/playlists/1G8IpkZKobrIlXcVPoSIuf/tracks?offset=100&limit=100&additional_types=track',
 'items': [{'added_at': '2020-11-29T15:02:07Z',
   'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/acclaimedmusic'},
    'href': 'https://api.spotify.com/v1/users/acclaimedmusic',
    'id': 'acclaimedmusic',
    'type': 'user',
    'uri': 'spotify:user:acclaimedmusic'},
   'is_local': False,
   'primary_color': None,
   'track': {'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1nJvji2KIlWSseXRSlNYsC'},
       'href': 'https://api.spotify.com/v1/artists/1nJvji2KIlWSseXRSlNYsC',
       'id': '1nJvji2KIlWSseXRSlNYsC',
       'name': 'The Velvet Underground',
       'type': 'artist',
       'uri': 'spotify:artist:1nJvji2KIlWSseXRSlNYsC'},
      {'external_urls': {'spotify': 'https://open.spotify.com/artist/0IwlY33zbBXN7zlS9DP2Cj'},
       'href': 'https://api.spotify.com/v1/artist

In [ ]:
#function to extract all songs from a large playlist

In [17]:
from random import randint
from time import sleep

def get_playlist_tracks(playlist_id):
    results = sp.user_playlist_tracks("spotify",playlist_id)
    tracks = results['items']
    while results['next']!=None:
        results = sp.next(results)
        tracks = tracks + results['items']
        sleep(randint(1,3))
    return tracks

In [18]:
all_tracks = get_playlist_tracks("1G8IpkZKobrIlXcVPoSIuf")
len(all_tracks)
#this now holds all 10,000 songs and their artists
#need to make it into a dataframe, once we've pared it down! 

10000

In [ ]:
#all_tracks holds all info for all 10,000 songs, it's big list of dictionary
#it's not fit to be a dataframe yet

In [ ]:
#we ONLY need artist, name of song, and URI for the database

Getting the artist name and song name

In [19]:
#getting to artist name
all_tracks[0]['track']['artists'][0]['name']

'Bob Dylan'

In [20]:
#getting to song name
all_tracks[0]['track']['name']

'Like a Rolling Stone'

### Getting the artist, song name, and URI from each song of my playlist

In [21]:
all_tracks[0]['track']['uri']

'spotify:track:3AhXZa8sUQht0UEdBJgpGc'

In [22]:
song_uri = all_tracks[0]['track']['uri']
#song_uri = all_tracks['items'][0]['track']['uri']
song_uri

'spotify:track:3AhXZa8sUQht0UEdBJgpGc'

In [23]:
sp.audio_features(song_uri)

[{'danceability': 0.482,
  'energy': 0.721,
  'key': 0,
  'loudness': -6.839,
  'mode': 1,
  'speechiness': 0.0321,
  'acousticness': 0.731,
  'instrumentalness': 0,
  'liveness': 0.189,
  'valence': 0.557,
  'tempo': 95.263,
  'type': 'audio_features',
  'id': '3AhXZa8sUQht0UEdBJgpGc',
  'uri': 'spotify:track:3AhXZa8sUQht0UEdBJgpGc',
  'track_href': 'https://api.spotify.com/v1/tracks/3AhXZa8sUQht0UEdBJgpGc',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3AhXZa8sUQht0UEdBJgpGc',
  'duration_ms': 369600,
  'time_signature': 4}]

In [24]:
#creating lists to hold song, artist, uri
song_name = []   
artist_name = []
track_uri = []

In [ ]:
#use a "for" loop to get the audio features, artists, and song names of all 10,000 songs in the playlist and
#put them inside the lists created above, making sure to ignore the Nans

In [25]:
for i in range(len(all_tracks)):
    try:
        song = all_tracks[i]['track']['name']
        artist = all_tracks[i]["track"]["album"]['artists'][0]['name']
        uri = all_tracks[i]['track']['uri']
        
        artist_name.append(artist)
        song_name.append(song)
        track_uri.append(uri)
    except:
        artist_name.append('none')
        song_name.append('none')
        track_uri.append('none')

In [28]:
spotify_df=pd.DataFrame(
    {'song': song_name,
     'artist': artist_name,
     'uri': track_uri
    })

In [29]:
spotify_df.head(5)

,song,artist,uri
0,Like a Rolling Stone,Bob Dylan,spotify:track:3AhXZa8sUQht0UEdBJgpGc
1,Smells Like Teen Spirit,Nirvana,spotify:track:3oTlkzk1OtrhH8wBAduVEi
2,A Day In The Life - Remastered,The Beatles,spotify:track:3ZFBeIyP41HhnALjxWy1pR
3,Good Vibrations (Mono),The Beach Boys,spotify:track:5Qt4Cc66g24QWwGP3YYV9y
4,Johnny B Goode,Chuck Berry,spotify:track:7MH2ZclofPlTrZOkPzZKhK


In [30]:
spotify_df.song.value_counts()
#can see there are 53 "nones"

none                    53
Summertime               6
Girlfriend               4
Crazy                    4
Fire                     4
                        ..
Young Adult Friction     1
The Pusher               1
If You Leave Me Now      1
Worried Life Blues       1
Come Dancing             1
Name: song, Length: 9641, dtype: int64

In [31]:
spotify_df = spotify_df[spotify_df.song != 'none'] # removing all the rows that have 'none'

In [32]:
spotify_df.shape

(9947, 3)

In [33]:
# since we removed all the values that had 'none' we need to reset index

spotify_df.reset_index(inplace=True)  

In [34]:
spotify_df.drop('index', axis=1, inplace=True) # drop extra index column created after reset_index

In [35]:
spotify_df.head()

,song,artist,uri
0,Like a Rolling Stone,Bob Dylan,spotify:track:3AhXZa8sUQht0UEdBJgpGc
1,Smells Like Teen Spirit,Nirvana,spotify:track:3oTlkzk1OtrhH8wBAduVEi
2,A Day In The Life - Remastered,The Beatles,spotify:track:3ZFBeIyP41HhnALjxWy1pR
3,Good Vibrations (Mono),The Beach Boys,spotify:track:5Qt4Cc66g24QWwGP3YYV9y
4,Johnny B Goode,Chuck Berry,spotify:track:7MH2ZclofPlTrZOkPzZKhK


### Getting the audio features

In [ ]:
#thanks to Asha and Erin for their help with this part!!

# collecting all the audiofeatures in a list
# commenting this code to prevent running the spotify api

# audio_features = []
# counter = 0  # setting a counter so that we can stop the code for 10 seconds when counter is 100

# for item in track_uri:
#     features = sp.audio_features(item)
#     audio_features.append(features)

#     counter = counter + 1
    
#     if counter % 100 == 0:
#         sleep(10)

In [36]:
sp.audio_features(track_uri[1])

[{'danceability': 0.485,
  'energy': 0.863,
  'key': 1,
  'loudness': -9.027,
  'mode': 1,
  'speechiness': 0.0495,
  'acousticness': 1.22e-05,
  'instrumentalness': 0.0162,
  'liveness': 0.138,
  'valence': 0.767,
  'tempo': 116.835,
  'type': 'audio_features',
  'id': '3oTlkzk1OtrhH8wBAduVEi',
  'uri': 'spotify:track:3oTlkzk1OtrhH8wBAduVEi',
  'track_href': 'https://api.spotify.com/v1/tracks/3oTlkzk1OtrhH8wBAduVEi',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3oTlkzk1OtrhH8wBAduVEi',
  'duration_ms': 300977,
  'time_signature': 4}]

In [40]:
# collecting all the audiofeatures in a list
# commenting this code to prevent running the spotify api

#audio_features = []
#counter = 0  # setting a counter so that we can stop the code for 10 seconds when counter is 100

#for item in track_uri:
#     features = sp.audio_features(item)
#     audio_features.append(features)

 #    counter = counter + 1
    
#     if counter % 100 == 0:
#         sleep(10)

In [41]:
[None] in audio_features # some of the audiofeatures have [None] because there was no trackuri

True

In [42]:
#creating lists to hold each audio feature

danceability = []
energy = []
loudness = []
speechiness = []
acousticness = []
instrumentalness = []
liveness = []
valence = []
tempo = []

In [43]:
for i in range(len(all_tracks)):
    try:
        danceability.append(audio_features[i][0]['danceability'])
        energy.append(audio_features[i][0]['energy'])
        loudness.append(audio_features[i][0]['loudness'])
        speechiness.append(audio_features[i][0]['speechiness'])
        acousticness.append(audio_features[i][0]['acousticness'])
        instrumentalness.append(audio_features[i][0]['instrumentalness'])
        liveness.append(audio_features[i][0]['liveness'])
        valence.append(audio_features[i][0]['valence'])
        tempo.append(audio_features[i][0]['tempo'])
    
    except:
        danceability.append('NaN')
        energy.append('NaN')
        loudness.append('NaN')        
        speechiness.append('NaN')
        acousticness.append('NaN')
        instrumentalness.append('NaN')
        liveness.append('NaN')
        valence.append('NaN')
        tempo.append('Na')

In [44]:
spotify_af = pd.DataFrame({'danceability' : danceability,
                          'energy': energy,
                          'loudness': loudness,
                          'speechiness': speechiness,
                           'acousticness': acousticness,
                           'instrumentalness': instrumentalness,
                           'liveness': liveness,
                           'valence': valence,
                           'tempo': tempo
                          })

In [45]:
spotify_af

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.482,0.721,-6.839,0.0321,0.731,0,0.189,0.557,95.263
1,0.485,0.863,-9.027,0.0495,0.000012,0.0162,0.138,0.767,116.835
2,0.364,0.457,-14.162,0.0675,0.29,0.000106,0.922,0.175,163.219
3,0.398,0.413,-10.934,0.0388,0.0822,0.000025,0.0891,0.331,133.574
4,0.518,0.756,-10.851,0.0915,0.735,0.000062,0.317,0.968,166.429
...,...,...,...,...,...,...,...,...,...
9995,0.332,0.707,-12.698,0.0361,0.000012,0.00612,0.11,0.652,144.815
9996,0.464,0.749,-8.564,0.48,0.224,0.000046,0.351,0.879,181.121
9997,0.694,0.71,-9.793,0.34,0.527,0.00135,0.0697,0.415,119.964
9998,0.332,0.8,-9.746,0.0326,0.000368,0.0016,0.085,0.832,149.24


In [46]:
spotify_final = pd.concat([spotify_df, spotify_af], axis=1)

In [ ]:
#afterwards, create a dataframe with all the features (and songs and artists?) and save in local drive?
#so don't need to re-do this whole process again if need to restart kernel 

In [47]:
spotify_final.head()

,song,artist,uri,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Like a Rolling Stone,Bob Dylan,spotify:track:3AhXZa8sUQht0UEdBJgpGc,0.482,0.721,-6.839,0.0321,0.731,0,0.189,0.557,95.263
1,Smells Like Teen Spirit,Nirvana,spotify:track:3oTlkzk1OtrhH8wBAduVEi,0.485,0.863,-9.027,0.0495,0.000012,0.0162,0.138,0.767,116.835
2,A Day In The Life - Remastered,The Beatles,spotify:track:3ZFBeIyP41HhnALjxWy1pR,0.364,0.457,-14.162,0.0675,0.29,0.000106,0.922,0.175,163.219
3,Good Vibrations (Mono),The Beach Boys,spotify:track:5Qt4Cc66g24QWwGP3YYV9y,0.398,0.413,-10.934,0.0388,0.0822,0.000025,0.0891,0.331,133.574
4,Johnny B Goode,Chuck Berry,spotify:track:7MH2ZclofPlTrZOkPzZKhK,0.518,0.756,-10.851,0.0915,0.735,0.000062,0.317,0.968,166.429


In [48]:
spotify_final.to_csv('spotify_final.csv')

# Unsupervised Machine Learning Lab | Clustering 

It's the moment to perform clustering on the songs you collected.

Remember that the ultimate goal of this little project is to improve the recommendations of artists. 

Clustering the songs will allow the recommendation system to limit the scope of the recommendations to only songs

that belong to the same cluster - songs with similar audio features.

In [ ]:
#import the dataset

In [ ]:
# The scale of "tempo" is much higher than the scale of many other variables!
# K-Means is a distance based algorithm: we need to scale / normalize:

from sklearn.preprocessing import StandardScaler
X_prep = StandardScaler().fit_transform(X_wine)

# Now, all features will have the same weight.
pd.DataFrame(X_prep,columns=X_wine.columns).head()